# Imports
###### This is all the imports that we use in the project

In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QMessageBox
from PyQt5.QtWidgets import QTableWidget,QTableWidgetItem
import nltk
import operator
import numpy as np
from nltk import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
wordlemmatizer = WordNetLemmatizer()
from string import punctuation
from nltk.corpus import stopwords
import math
from numpy import dot
from numpy.linalg import norm
import re
from rake_nltk import Rake
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
from collections import Counter
import os
import rbm
import webbrowser
import glob
import matplotlib.pyplot as plt 
import timeit

import img_rc
import uploadImg_rc
import similarimg_rc
import saveimgnew_rc
import sim_rc

import spacy 
nlp = spacy.load('en_core_web_lg') 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\no_ha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\no_ha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\no_ha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\no_ha\anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implem

# save uploaded text to folder 
###### Saves the uploaded text file to the project folder

In [2]:
def saveTextToFolder(text,fileNameOnly):
   
    txtName=fileNameOnly+".txt"    
    text_file = open(txtName, "w")
    text_file.write(text)
    text_file.close()

# Fuse
###### This function concatenates 2 summaries together

In [3]:
def fuseSummariesFunc(text,previoustext):
    #Open files and extract strings
    with open(text, 'r') as file:
        textFileSTR = file.read().replace('\n', '')
    with open(previoustext, 'r') as file:
        previoustextFileSTR = file.read().replace('\n', '') 
    #Get new fileName
    file_name_fix1=text.replace('_Summary.txt','')
    file_name_fix2=previoustext.replace('_Summary.txt','')
    SummaryName=file_name_fix1+"_"+file_name_fix2+"_Summary.txt"
    #Write to file
    text_file = open(SummaryName, "w")
    text_file.write(textFileSTR)
    text_file.write("\n")
    text_file.write("\n")
    text_file.write(previoustextFileSTR)
    text_file.close()
    #Save Concatenated original texts
    txtOriginal1=file_name_fix1+".txt"
    txtOriginal2=file_name_fix2+".txt"
    with open(txtOriginal1, 'r') as file:
        original1 = file.read().replace('\n', '')
    with open(txtOriginal2, 'r') as file:
        original2 = file.read().replace('\n', '')
    Local_Dictionary(original1+original2,file_name_fix1+"_"+file_name_fix2)
    txtName=file_name_fix1+"_"+file_name_fix2+".txt" 
    text_original_fuse = open(txtName, "w")
    text_original_fuse.write(original1)
    text_original_fuse.write("\n")
    text_original_fuse.write("\n")
    text_original_fuse.write(original2)
    text_original_fuse.close()
    
    #Allert box asks if you want to open the new file
    msgBox = QtWidgets.QMessageBox( QtWidgets.QMessageBox.Question, "New summary created", "Would you like to open the file ?", QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No )
    result = msgBox.exec_()
    if result == QtWidgets.QMessageBox.Yes:
        webbrowser.open(SummaryName)

# Check Similiarity Function 
###### This function looks for similiar summaries in the folder

In [4]:
def checkForSimilarSummaries(text):
    #Create graph pictures directory
    try:
        # Create target Directory
        os.mkdir('graphPictures')
        print("Directory Created ") 
    except FileExistsError:
        print("Directory already exists")  
    ScoreList=[]
    #This is the function that searches for similar summaries
    NumberOfWords=5
    NumberOfWords2=NumberOfWords+1
    #Open Source file
    with open(text, 'r') as file:
        sourceFile = file.read().replace('\n', '')   
    #Prep the dictionary function
    def text_prep(data):  
        data=data.replace('(', '')
        data=data.replace(')', '')
        data=data.replace(',', '')
        data=data.replace("'", "")
        data = ''.join([i for i in data if not i.isdigit()])
        data=data.split()[:NumberOfWords2]
        return data
    #Get all dictionaries from folder
    sourceFileList=text_prep(sourceFile)
    print(sourceFileList[0])
    saved_dictionaries=glob.glob("*_Dictionary.txt")
    saved_dictionaries.remove(text)
    print(saved_dictionaries)
    #Run on dictionaries
    for saved_dict in saved_dictionaries:
        file_name_fix=saved_dict.replace('_Dictionary.txt','')
        #Open dictionaries
        with open(saved_dict, 'r') as file:
            targetFile = file.read().replace('\n', '')
        targetFileList=text_prep(targetFile)
        similaritySum=0
        #Run on every word of source with words of dictionary
        for i in range(NumberOfWords2):
            targetFileListString = ' '.join(targetFileList)
            targetFileListString = sourceFileList[i]+" "+targetFileListString
            tokens = nlp(targetFileListString)
            #Calculate Score
            left = [1, 2, 3, 4, 5]  # x-coordinates of left sides of bars  
            height = []
            tick_label = []
            for j in range(NumberOfWords):
                similaritySum = similaritySum + tokens[0].similarity(tokens[1+j])
                height.append(tokens[0].similarity(tokens[1+j]))
                tick_label.append(tokens[1+j].text)   
            plt.bar(left, height, tick_label = tick_label, width = 0.4, color =['blue'])
            plt.xlabel('Words') # naming the x-axis 
            plt.ylabel('similarity Score') # naming the y-axis 
            plt.title('Similarity graph for word : '+ tokens[0].text) # plot title 
            picString = 'graphPictures/'+file_name_fix+'_'+tokens[0].text+'_graph.png'
            plt.savefig(picString) # function to save the plot 
            #plt.show()
        similaritySum=round(similaritySum,3)
        ScoreListItem=file_name_fix+" "+str(similaritySum)
        ScoreList.append(ScoreListItem) # Save the score to list
    return ScoreList #Return score list

# Dictionary
###### Create local dictionary 

In [5]:
def Local_Dictionary(text,fileNameOnly):
    def lemmatize_words(words):
        lemmatized_words = []
        for word in words:
            lemmatized_words.append(wordlemmatizer.lemmatize(word))
        return lemmatized_words
    def freq(words):
        words = [word.lower() for word in words]
        dict_freq = {}
        words_unique = []
        for word in words:
            if word not in words_unique:
                words_unique.append(word)
        for word in words_unique:
            dict_freq[word] = words.count(word)
        return dict_freq
    def remove_special_characters(text):
        regex = r'[^a-zA-Z0-9\s]'
        text = re.sub(regex,'',text)
        return text
    Stopwords = set(stopwords.words('english'))
    text = remove_special_characters(str(text)) #remove special characters
    text = re.sub(r'\d+', '', text)             
    tokenized_words_with_stopwords = word_tokenize(text) #Tokenize the words
    tokenized_words = [word.lower() for word in tokenized_words_with_stopwords]
    tokenized_words = [word for word in tokenized_words if word not in Stopwords and word not in punctuation and not word.isdigit()]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = lemmatize_words(tokenized_words)
    new_Vocabulary = freq(tokenized_words) # Look for the word frequancy and create dictionary
    dictionary_File_Name=fileNameOnly+"_Dictionary.txt"
    dict_file = open(dictionary_File_Name, "w") # Create dictionary file
    # Iterate over key/value pairs in dict and print them
    new_vocabulary_sorted = sorted(new_Vocabulary.items(), key=operator.itemgetter(1))
    new_vocabulary_sorted.reverse()
    with open(dictionary_File_Name, "w") as outfile:
        outfile.write("\n".join(str(item) for item in new_vocabulary_sorted))

# Algorithm
###### This is the main algorithm that is responsible for summarizing the texts

In [6]:
def Summarize(text,percentageUser,RbmAmount,fileNameOnly):
    start_timeF = timeit.default_timer()
    sentences=(sent_tokenize(text))
    emptyarray= np.empty((len(sentences),1,3),dtype=object)
    # # Create empty array # #
    for s in range(len(sentences)):
        emptyarray[s][0][0] = sentences[s]
        emptyarray[s][0][1] = s
# # Tokenization, Stop word removal , Bi-grams, Tri-grams------------------------------------------------------
    stop_words = set(stopwords.words('english'))
    bi_token=[]
    bi_token_length=[]
    tri_token_length=[]
    for u in range(len(sentences)):
        sent_split1=[w.lower() for w in sentences[u].split(" ")]
        sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
        bigrams_list = [bigram for bigram in nltk.bigrams(sent_split)]
        bi_token.append(bigrams_list)
        bi_token_length.append(len(bi_token[u]))
    bi_tokens = [(int(o) / max(bi_token_length))*100 for o in bi_token_length]
    tri_token=[]
    for u in range(len(sentences)):
        sent_split2=[w.lower() for w in sentences[u].split(" ")]
        sent_split3=[w for w in sent_split2 if w not in stop_words and w not in punctuation and not w.isdigit()]
        trigrams_list = [trigram for trigram in nltk.trigrams(sent_split3)]
        tri_token.append(trigrams_list)
        tri_token_length.append(len(tri_token[u]))
    tri_tokens = [(int(m) / max(tri_token_length))*100 for m in tri_token_length]
# # Sentence Position Feature-----------------------------------------------------------------------------------
    def position(l):
        return [index for index, value in enumerate(sentences)]
    sent_position= (position(sentences))
    num_sent=len(sent_position)
    position = []
    position_rbm = []
    sent_pos1_rbm = 1
    sent_pos1 = 100
    position.append(sent_pos1)
    position_rbm.append(sent_pos1_rbm)
    for x in range(1,num_sent-1):
        s_p= ((num_sent-x)/num_sent)*100
        position.append(s_p)
        s_p_rbm = (num_sent-x)/num_sent
        position_rbm.append(s_p_rbm)   
    sent_pos2 = 100
    sent_pos2_rbm = 1
    position.append(sent_pos2)
    position_rbm.append(sent_pos2_rbm)
# # Converting Sentences to Vectors-------------------------------------------------------------------------------
    def convertToVSM(sentences):
        vocabulary = []
        for sents in sentences:
            vocabulary.extend(sents)
        vocabulary = list(set(vocabulary))
        vectors = []
        for sents in sentences:
            vector = []
            for tokenss in vocabulary:
                vector.append(sents.count(tokenss))
            vectors.append(vector)
        return vectors
    VSM=convertToVSM(sentences)
# # TF-IDF---------------------------------------------------------------------------------------------------------
    sentencelength=len(sentences)
    def calcMeanTF_ISF(VSM, index):
        vocab_len = len(VSM[index])
        sentences_len = len(VSM)
        count = 0
        tfisf = 0
        for i in range(vocab_len):
            tf = VSM[index][i]
            if(tf>0):
                count += 1
                sent_freq = 0
                for j in range(sentences_len):
                    if(VSM[j][i]>0): sent_freq += 1
                tfisf += (tf)*(1.0/sent_freq)
        if(count > 0):
            mean_tfisf = tfisf/count
        else:
            mean_tfisf = 0
        return tf, (1.0/sent_freq), mean_tfisf
    tfvec=[]
    isfvec=[]
    tfisfvec=[]
    tfisfvec_rbm=[]
    for i in range(sentencelength):
        x,y,z=calcMeanTF_ISF(VSM,i)
        tfvec.append(x)
        isfvec.append(y)
        tfisfvec.append(z*100)
        tfisfvec_rbm.append(z)
    maxtf_isf=max(tfisfvec_rbm)
    centroid=[]
    centroid.append(maxtf_isf)
    centroid=(max(VSM))
    def add_noise(new_word_rep):
        c=0
        new_word_rep1=[]
        noise=np.random.normal(0,0.1,len(new_word_rep))
        for word in new_word_rep:
            new_word_rep1.append(new_word_rep[c]+noise[c])
            #print(noise[c])
            c=c+1
        return new_word_rep1
# # Cosine Similarity between Centroid and Sentences---------------------------------------------------------------
    cosine_similarity=[]
    cosine_similarity_rbm=[]
    for z in range(sentencelength):
        cos_simi = ((dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))*100)
        cosine_similarity.append(cos_simi)
        cos_simi_rbm = (dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))
        cosine_similarity_rbm.append(cos_simi_rbm)
# # Sentence length feature------------------------------------------------------------------------------------------
    sent_word=[]
    for u in range(len(sentences)):
        sent_split1=[w.lower() for w in sentences[u].split(" ")]
        sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
        a=(len(sent_split))
        sent_word.append(a)
    longest_sent=max(sent_word)
    sent_length=[]
    sent_length_rbm=[]
    for x in sent_word:
        sent_length.append((x/longest_sent)*100)
        sent_length_rbm.append(x/longest_sent)
# # Numeric token Feature----------------------------------------------------------------------------------------------
    num_word=[]
    numeric_token=[]
    numeric_token_rbm=[]
    for u in range(len(sentences)):
        sent_split4=sentences[u].split(" ")
        e=re.findall("\d+",sentences[u])
        noofwords=(len(e))
        num_word.append(noofwords)
        try:
            numeric_token.append((num_word[u]/sent_word[u])*100)
            numeric_token_rbm.append(num_word[u]/sent_word[u])
        except:
            numeric_token.append((num_word[u]/(sent_word[u]+1)*100))
            numeric_token_rbm.append(num_word[u]/(sent_word[u]+1))
# # Thematic words feature--------------------------------------------------------------------------------------
    r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
    keywords=[]
    # If you want to provide your own set of stop words and punctuations to
    # r = Rake(<list of stopwords>, <string of puntuations to ignore>)
    for s in sentences:
        r.extract_keywords_from_text(s)
        key=list(r.get_ranked_phrases())
        keywords.append(key)
    l_keywords=[]
    for s in keywords:
        leng=len(s)
        l_keywords.append(leng)
    total_keywords=sum(l_keywords)
    thematic_number= []
    thematic_number_rbm= []
    for x in l_keywords:
        try:
            thematic_number.append((x/total_keywords)*100)
            thematic_number_rbm.append(x/total_keywords)
        except:
            thematic_number.append((x/total_keywords+0.1)*100)
            thematic_number_rbm.append(x/total_keywords+0.1)
# # proper noun feature---------------------------------------------------------------------------------------
    pncounts = []
    pncounts_rbm = []
    for sentence in sentences:
        tagged=nltk.pos_tag(nltk.word_tokenize(str(sentence)))
        counts = Counter(tag for word,tag in tagged if tag.startswith('NNP') or tag.startswith('NNPS'))
        f=sum(counts.values())
        pncounts.append(f)
        pncounts_rbm.append(f)
    try:
        pnounscore=[(int(o) / int(p))*100 for o,p in zip(pncounts, sent_word)]
        pnounscore_rbm=[int(o) / int(p) for o,p in zip(pncounts_rbm, sent_word)]
    except:
        pnounscore=[(int(o) / (0.1+int(p)))*100 for o,p in zip(pncounts, sent_word)]
        pnounscore_rbm=[int(o) / (0.1+int(p)) for o,p in zip(pncounts_rbm, sent_word)]
# # feature matrix -----------------------------------------------------------------------------------------
    def matrixRbm(tfisfvec_rbm) :
        featureMatrix = []
        featureMatrix.append(position_rbm)
        featureMatrix.append(bi_token_length)
        featureMatrix.append(tri_token_length)
        featureMatrix.append(tfisfvec_rbm)
        featureMatrix.append(cosine_similarity_rbm)
        featureMatrix.append(thematic_number_rbm)
        featureMatrix.append(sent_length_rbm)
        featureMatrix.append(numeric_token_rbm)
        featureMatrix.append(pnounscore_rbm)
        featureMat = np.zeros((len(sentences),9))
        for i in range(9) :
            for j in range(len(sentences)):
                featureMat[j][i] = featureMatrix[i][j]
        featureMat_normed = featureMat
        print(featureMat_normed)
        return featureMat_normed
    def sumRbmMat(temp):
        enhanced_feature_sum = []
        for i in range(len(np.sum(temp,axis=1))) :
            enhanced_feature_sum.append([np.sum(temp,axis=1)[i],i])
            emptyarray[i][0][2]=np.sum(temp,axis=1)[i]
        return enhanced_feature_sum 
#RBM----------------------------------------------------------------------------------------
    prcSummary=int(percentageUser)
    rbmNum=int(RbmAmount)
    numberOfEnsambleRBM=rbmNum
    noiseMat=[]
    for i in range(rbmNum):
        noiseMat.append(add_noise(tfisfvec_rbm))
    rbmMat=[]
    for i in range(len(noiseMat)):
        print("\n")
        rbmMat.append(matrixRbm(noiseMat[i]))
    tempPostRbm=[]
    postRbm=[]
    end_timeF = timeit.default_timer()
    pretraining_timeF = (end_timeF - start_timeF)
    for i in range(len(noiseMat)):
        tempPostRbm.append(rbm.test_rbm(dataset = rbmMat[i],learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9,n_hidden=9))
        for j in range(numberOfEnsambleRBM-1):
            tempPostRbm.append(rbm.test_rbm(dataset = tempPostRbm[j][0],learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9,n_hidden=9))
        #print("\n\nRBM Output2\n\n")
        #print("\n\n"+str(numberOfEnsambleRBM)+"\n\n")
        #print(tempPostRbm[(numberOfEnsambleRBM-1)+(i*numberOfEnsambleRBM)][0])
        postRbm.append(tempPostRbm[(numberOfEnsambleRBM-1)+(i*numberOfEnsambleRBM)][0])
    if(int(RbmAmount)==3):
        total_time=round(pretraining_timeF/60,8)+round(tempPostRbm[0][2]/60,8)+round(tempPostRbm[1][2]/60,8)+round(tempPostRbm[2][2]/60,8)
        msg = QMessageBox()
        msg.setMaximumHeight(1000)
        msg.setWindowTitle("Training")
        msg.setText("Training Complete:"+ "\n    Time of word processing : "+ str(round(pretraining_timeF/60,8)) + " seconds"
                       "\n    Number of Training epochs for RBM ensemble # 1 : "+ str(tempPostRbm[0][1])+"   " +
                       "\n    Training of RBM ensemble #1 took : "+str(round(tempPostRbm[0][2]/60,8))+" seconds"+"         " +
                       "\n    Number of Training epochs for RBM ensemble # 2 : "+ str(tempPostRbm[1][1]) +"   " +
                       "\n    Training of RBM ensemble #2 took : "+str(round(tempPostRbm[1][2]/60,8))+" seconds"+"         " +
                       "\n    Number of Training epochs for RBM ensemble # 3 : "+ str(tempPostRbm[2][1]) +"   " +
                       "\n    Training of RBM ensemble #3 took : "+str(round(tempPostRbm[2][2]/60,8))+" seconds"+"         "+
                       "\n Total time : "+str(round(total_time,8))+" seconds") 
    if(int(RbmAmount)==1):
        total_time=round(pretraining_timeF/60,8)+round(tempPostRbm[0][2]/60,8)
        msg = QMessageBox()
        msg.setWindowTitle("Training")
        msg.setText("Training Complete:"+ "\n    Time of word processing : "+ str(round(pretraining_timeF/60,8)) + " seconds" 
                        "\n    Number of Training epochs for RBM ensemble # 1 : "+ str(tempPostRbm[0][1]) +
                        "\n    Training of RBM ensemble #1 took : "+str(round(tempPostRbm[0][2]/60,8))+" seconds"+"         "+
                        "\n Total time : "+str(round(total_time,8))+" seconds")
    if(int(RbmAmount)==5):
        total_time=round(pretraining_timeF/60,8)+round(tempPostRbm[0][2]/60,8)+round(tempPostRbm[1][2]/60,8)+round(tempPostRbm[2][2]/60,8)+round(tempPostRbm[3][2]/60,8)+round(tempPostRbm[4][2]/60,8)
        msg = QMessageBox()
        msg.setWindowTitle("Training")
        msg.setText("Training Complete:"+ "\n    Time of word processing : "+ str(round(pretraining_timeF/60,8)) + " seconds"
                       "\n    Number of Training epochs for RBM ensemble # 1 : "+ str(tempPostRbm[0][1]) +
                       "\n    Training of RBM ensemble #1 took : "+str(round(tempPostRbm[0][2]/60,8))+" seconds"+"         "+
                       "\n    Number of Training epochs for RBM ensemble# 2 : "+ str(tempPostRbm[1][1]) +
                       "\n    Training of RBM ensemble #2 took : "+str(round(tempPostRbm[1][2]/60,8))+" seconds"+"         "+
                       "\n    Number of Training epochs for RBM ensemble # 3 : "+ str(tempPostRbm[2][1]) +
                       "\n    Training of RBM ensemble #3 took : "+str(round(tempPostRbm[2][2]/60,8))+" seconds"+"         "+
                       "\n    Number of Training epochs for RBM ensemble # 4 : "+ str(tempPostRbm[3][1]) +
                       "\n    Training of RBM ensemble #4 took : "+str(round(tempPostRbm[3][2]/60,8))+" seconds"+"         "+
                       "\n    Number of Training epochs for RBM ensemble # 5 : "+ str(tempPostRbm[4][1]) +
                       "\n    Training of RBM ensemble #5 took : "+str(round(tempPostRbm[4][2]/60,8))+" seconds"+"         "+
                       "\n Total time : "+str(round(total_time,8))+" seconds")
                     
                     
    x = msg.exec_()  # this will show our messagebox
    enhanced_feature_sum =[]
    
#Next Step Post RBM------------------------------------------------
    for i in range(len(noiseMat)):
        enhanced_feature_sum.append(sumRbmMat(postRbm[i]))
    for i in range(len(noiseMat)):
        enhanced_feature_sum[i].sort(key=lambda x: x[1])
    topMat= [[0 for i in range(2)] for j in range(len(enhanced_feature_sum[0]))] # create empty matrix
    # Sum rankings of all rbms
    for i in range(len(enhanced_feature_sum[0])):
        topMat[i][1]=i
        for k in range(len(enhanced_feature_sum)):
            topMat[i][0]=topMat[i][0]+enhanced_feature_sum[k][i][0] 
#Final step , Save summary into file and select the top most sentences--------
    SummaryName=fileNameOnly+"_Summary.txt"
    text_file = open(SummaryName, "w")
    topMat.sort(key=lambda x: x[0])
    Sentences_to_Print=round((prcSummary/100)*num_sent)
    print(Sentences_to_Print)
    extractedSentences=[]
    j= (len(topMat))-1
    for i in range(Sentences_to_Print):
        extractedSentences.append(topMat[j][1])
        j=j-1
    extractedSentences.sort()
    print(extractedSentences)
    for i in range(Sentences_to_Print):
        print(sentences[extractedSentences[i]])
        text_file.write(sentences[extractedSentences[i]])
        text_file.write("\n")
    text_file.close()

# Graph window
###### This is the GUI and Controller of the graph window

In [7]:
class Ui_similar_graph(object):
    def setupUi(self, similar_graph,clickedFileName,originialSource):
        similar_graph.setObjectName("similar_graph")
        similar_graph.resize(1099, 976)
        self.centralwidget = QtWidgets.QWidget(similar_graph)
        self.centralwidget.setObjectName("centralwidget")
        
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(370, 5, 421, 61))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(20)
        font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(75)
        font.setStrikeOut(False)
        self.label.setFont(font)
        self.label.setMouseTracking(False)
        self.label.setObjectName("label")
        
        #self.label.setGeometry(QtCore.QRect(360, 20, 571, 41))
        self.word3 = QtWidgets.QLabel(self.centralwidget)
        self.word3.setGeometry(QtCore.QRect(40, 678, 481, 291))
        self.word3.setFrameShape(QtWidgets.QFrame.Panel)
        self.word3.setText("")
        self.word3.setObjectName("word3")
        self.word4 = QtWidgets.QLabel(self.centralwidget)
        self.word4.setGeometry(QtCore.QRect(580, 78, 481, 291))
        self.word4.setFrameShape(QtWidgets.QFrame.Panel)
        self.word4.setText("")
        self.word4.setObjectName("word4")
        self.word1 = QtWidgets.QLabel(self.centralwidget)
        self.word1.setGeometry(QtCore.QRect(40, 78, 481, 291))
        self.word1.setFrameShape(QtWidgets.QFrame.Panel)
        self.word1.setText("")
        self.word1.setObjectName("word1")
        self.word5 = QtWidgets.QLabel(self.centralwidget)
        self.word5.setGeometry(QtCore.QRect(580, 378, 481, 291))
        self.word5.setFrameShape(QtWidgets.QFrame.Panel)
        self.word5.setFrameShadow(QtWidgets.QFrame.Plain)
        self.word5.setText("")
        self.word5.setObjectName("word5")
        self.word2 = QtWidgets.QLabel(self.centralwidget)
        self.word2.setGeometry(QtCore.QRect(40, 378, 481, 291))
        self.word2.setFrameShape(QtWidgets.QFrame.Panel)
        self.word2.setText("")
        self.word2.setObjectName("word2")
        self.word6 = QtWidgets.QLabel(self.centralwidget)
        self.word6.setGeometry(QtCore.QRect(580, 678, 481, 291))
        self.word6.setFrameShape(QtWidgets.QFrame.Panel)
        self.word6.setText("")
        self.word6.setObjectName("word6")
        similar_graph.setCentralWidget(self.centralwidget)
        
        #ClickedFileName
        #originialSource
        
        
        file_name_fix=originialSource.replace('_Summary.txt','')
        originialSourceString=file_name_fix+"_Dictionary.txt"
        with open(originialSourceString, 'r') as file:
            data = file.read().replace('\n', '') 
        data=data.replace('(', '')
        data=data.replace(')', '')
        data=data.replace(',', '')
        data=data.replace("'", "")
        data = ''.join([i for i in data if not i.isdigit()])
        data=data.split()[:6]
        #sourceFileList=textprep(sourceFile)
        self.pixmanString1='graphPictures/'+clickedFileName+'_'+data[0]+'_graph.png'
        self.pixmanString2='graphPictures/'+clickedFileName+'_'+data[1]+'_graph.png'
        self.pixmanString3='graphPictures/'+clickedFileName+'_'+data[2]+'_graph.png'
        self.pixmanString4='graphPictures/'+clickedFileName+'_'+data[3]+'_graph.png'
        self.pixmanString5='graphPictures/'+clickedFileName+'_'+data[4]+'_graph.png'
        self.pixmanString6='graphPictures/'+clickedFileName+'_'+data[5]+'_graph.png'


        
        #graph1
        pixmap = QtGui.QPixmap(self.pixmanString1) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word1.width(), self.word1.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word1.setPixmap(pixmap) # Set the pixmap onto the label
        self.word1.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        #graph2
        pixmap = QtGui.QPixmap(self.pixmanString2) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word2.width(), self.word2.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word2.setPixmap(pixmap) # Set the pixmap onto the label
        self.word2.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        #graph3
        pixmap = QtGui.QPixmap(self.pixmanString3) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word3.width(), self.word3.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word3.setPixmap(pixmap) # Set the pixmap onto the label
        self.word3.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        #graph4
        pixmap = QtGui.QPixmap(self.pixmanString4) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word4.width(), self.word4.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word4.setPixmap(pixmap) # Set the pixmap onto the label
        self.word4.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        #graph5
        pixmap = QtGui.QPixmap(self.pixmanString5) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word5.width(), self.word5.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word5.setPixmap(pixmap) # Set the pixmap onto the label
        self.word5.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        #graph6
        pixmap = QtGui.QPixmap(self.pixmanString6) # Setup pixmap with the provided image
        pixmap = pixmap.scaled(self.word6.width(), self.word6.height(), QtCore.Qt.KeepAspectRatio) # Scale pixmap
        self.word6.setPixmap(pixmap) # Set the pixmap onto the label
        self.word6.setAlignment(QtCore.Qt.AlignCenter) # Align the label to center
        
        
        
        self.retranslateUi(similar_graph)
        QtCore.QMetaObject.connectSlotsByName(similar_graph)

        
   
    def retranslateUi(self, similar_graph):
        _translate = QtCore.QCoreApplication.translate
        similar_graph.setWindowTitle(_translate("similar_graph", "Graphs"))
        self.label.setText(_translate("Upload_Window", "Word similarity graphs "))

# Upload file window
###### This is the GUI and Controller of the Upload file window

In [8]:
class Ui_Upload_Window(object):
    def setupUi(self, Upload_Window):
        Upload_Window.setObjectName("Upload_Window")
        Upload_Window.resize(800, 600)
        self.Upload_Window_back=Upload_Window
        self.centralwidget = QtWidgets.QWidget(Upload_Window)
        self.centralwidget.setObjectName("centralwidget")
        
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(0, -10, 811, 631))
        self.label.setStyleSheet("background-image: url(:/newPrefix/upload.jpg);")
        self.label.setText("")
        self.label.setObjectName("label")
        
        
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(80, 40, 641, 71))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(28)
        font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(60)
        font.setStrikeOut(False)
        self.label.setFont(font)
        self.label.setMouseTracking(False)
        self.label.setObjectName("label")
        self.BackButton = QtWidgets.QPushButton(self.centralwidget)
        self.BackButton.setGeometry(QtCore.QRect(22, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.BackButton.setFont(font)
        self.BackButton.setObjectName("BackButton")
        self.HelpButton = QtWidgets.QPushButton(self.centralwidget)
        self.HelpButton.setGeometry(QtCore.QRect(680, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.HelpButton.setFont(font)
        self.HelpButton.setObjectName("HelpButton")
        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(300, 110, 151, 28))
        
        
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.pushButton.setFont(font)
        self.pushButton.setObjectName("pushButton")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(100, 160, 231, 20))
        self.label_2.setObjectName("label_2")
        self.comboBox = QtWidgets.QComboBox(self.centralwidget)
        self.comboBox.setGeometry(QtCore.QRect(310, 160, 73, 22))
        self.comboBox.setObjectName("comboBox")
        self.comboBox.addItem("10") 
        self.comboBox.addItem("30") 
        self.comboBox.addItem("50") 
        self.comboBox.addItem("70")
        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setGeometry(QtCore.QRect(410, 160, 161, 20))
        self.label_3.setObjectName("label_3")
        self.comboBox2 = QtWidgets.QComboBox(self.centralwidget)
        self.comboBox2.setGeometry(QtCore.QRect(570, 160, 73, 22))
        self.comboBox2.setObjectName("comboBox2")
        self.comboBox2.addItem("3") 
        self.comboBox2.addItem("1") 
        self.comboBox2.addItem("5") 
        self.SummarizeButton = QtWidgets.QPushButton(self.centralwidget)
        self.SummarizeButton.setGeometry(QtCore.QRect(270, 200, 211, 41))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.CheckButton = QtWidgets.QPushButton(self.centralwidget)
        self.CheckButton.setGeometry(QtCore.QRect(500, 510, 161, 28))
        self.CheckButton.setObjectName("CheckButton")
        self.OpenVocabularyButton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenVocabularyButton.setGeometry(QtCore.QRect(350, 510, 141, 28))
        self.OpenVocabularyButton.setObjectName("OpenVocabularyButton")
        self.tableWidget = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget.setGeometry(QtCore.QRect(90, 260, 571, 231))
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(1)
        self.tableWidget.setRowCount(0)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(0, item)
        self.tableWidget.setEditTriggers(QtWidgets.QAbstractItemView.NoEditTriggers)
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(13)
        font.setBold(True)
        self.SummarizeButton.setFont(font)
        self.SummarizeButton.setObjectName("SummarizeButton")
        
        header = self.tableWidget.horizontalHeader()
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        
        self.OpenSummaryButton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSummaryButton.setGeometry(QtCore.QRect(220, 510, 121, 28))
        self.OpenSummaryButton.setObjectName("OpenSummaryButton")
        
        self.OpenSource = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSource.setGeometry(QtCore.QRect(100, 510, 101, 28))
        self.OpenSource.setObjectName("Open Source")
        
        Upload_Window.setCentralWidget(self.centralwidget)

        self.retranslateUi(Upload_Window)
        QtCore.QMetaObject.connectSlotsByName(Upload_Window)      
        
        #Veriables
        self.text=""
        self.fileNameOnly=""
        self.tableValue=""
        self.tableIndex=0
        
        if(self.tableValue==""):
            self.OpenSummaryButton.setDisabled(True)
            self.OpenVocabularyButton.setDisabled(True)
            self.CheckButton.setDisabled(True)
            self.SummarizeButton.setDisabled(True)
            self.comboBox.setDisabled(True)
            self.comboBox2.setDisabled(True)
            self.OpenSource.setDisabled(True)
            
        #Button Functions
        QtCore.QMetaObject.connectSlotsByName(Upload_Window)
        self.BackButton.clicked.connect(self.Back_toMain_Func)
        self.HelpButton.clicked.connect(self.Help_Func)
        self.pushButton.clicked.connect(self.upload_File)
        self.SummarizeButton.clicked.connect(self.summarize_Text)
        self.tableWidget.clicked.connect(self.TableContent)
        self.OpenSummaryButton.clicked.connect(self.openSummaryFile)
        self.OpenVocabularyButton.clicked.connect(self.openVocabFile)
        self.CheckButton.clicked.connect(self.checkForSimiliarSummaries)
        self.OpenSource.clicked.connect(self.OpenSourceFunc)

    def checkForSimiliarSummaries(self):
        dictName=self.tableValue+"_Dictionary.txt"
        list_init=checkForSimilarSummaries(dictName)
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Similar_Summaries()
        self.ui.setupUi(self.window,list_init,self.tableValue)
        self.window.show()
    def openVocabFile(self):
        dictName=self.tableValue+"_Dictionary.txt"
        webbrowser.open(dictName)
    def openSummaryFile(self):
        SummaryName=self.tableValue+"_Summary.txt"
        webbrowser.open(SummaryName)
    def TableContent(self):
        # #selected cell value.
        index=(self.tableWidget.selectionModel().currentIndex())
        self.OpenSummaryButton.setDisabled(False)
        self.OpenVocabularyButton.setDisabled(False)
        self.CheckButton.setDisabled(False)
        self.OpenSource.setDisabled(False)
        # print(index)
        self.tableValue=index.sibling(index.row(),index.column()).data()
    def Back_toMain_Func(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Opening_Window()
        self.ui.setupUi(self.window)
        self.Upload_Window_back.hide()
        self.window.show()
    def Help_Func(self):
        msg = QMessageBox()
        msg.setWindowTitle("Help")
        msg.setText("If you found a bug or want to ask any question "+"\nyou can contact us at:\n"
        + "Nofar Hazan- nofarhazan5@gmail.com\nDaniel Larionov-danielarionov@gmail.com ")
        x = msg.exec_()  # this will show our messagebox
    def upload_File(self):
        fileName, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select File", "", "Txt Files (*.txt)")
        print(fileName)
        ext = fileName.split('/')
        x=ext[len(ext)-1]
        txt=".txt"
        self.fileNameOnly = x[:x.find(txt)]
        print(self.fileNameOnly)
        
        saved_Summaries=glob.glob("*_Summary.txt")
        summarySearch=self.fileNameOnly+"_Summary.txt"
        for file in saved_Summaries:
             if (file == summarySearch):
                msg = QMessageBox()
                msg.setWindowTitle("Warning")
                msg.setText("You are about to change "+'"'+self.fileNameOnly+'"'+" an existing summary!")
                x = msg.exec_()  # this will show our messagebox          
        self.text=open(fileName).read()
        self.comboBox.setDisabled(False)
        self.comboBox2.setDisabled(False)
        self.SummarizeButton.setDisabled(False)
    def summarize_Text(self,text):
        comboBoxText = str(self.comboBox.currentText())
        comboBoxText2 = str(self.comboBox2.currentText())
        print("\n\n\n")
        Local_Dictionary(self.text,self.fileNameOnly)
        msg = QMessageBox()
        msg.setWindowTitle("Training Begins")
        msg.setText("Training process begins please wait untill completion")
        x = msg.exec_()  # this will show our messagebox
        saveTextToFolder(self.text,self.fileNameOnly)
        Summarize(self.text,comboBoxText,comboBoxText2,self.fileNameOnly)
        dictName=self.fileNameOnly+"_Dictionary.txt"
        #Add line on table
        rowPosition = self.tableWidget.rowCount()
        self.tableWidget.insertRow(rowPosition)
        self.tableWidget.setItem(0,self.tableIndex, QTableWidgetItem(self.fileNameOnly))
        self.tableIndex=self.tableIndex+1
        
    def OpenSourceFunc(self):
        flag=0
        #SummaryName=self.tableValue+".txt"
        saved_Summaries=glob.glob("*.txt")
        summarySearch=self.tableValue+".txt"
        for file in saved_Summaries:
            if (file == summarySearch):
                webbrowser.open(summarySearch)
                flag=1
        if (flag==0):
            fileName, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select File", "", "Txt Files (*.txt)")
            webbrowser.open(fileName)
        
    def retranslateUi(self, Upload_Window):
        _translate = QtCore.QCoreApplication.translate
        Upload_Window.setWindowTitle(_translate("Upload_Window", "Upload file"))
        self.label.setText(_translate("Upload_Window", "Automatic Text Summarization"))
        self.BackButton.setText(_translate("Upload_Window", "Back"))
        self.HelpButton.setText(_translate("Upload_Window", "Help"))
        self.pushButton.setText(_translate("Upload_Window", "Upload File/s"))
        self.label_2.setText(_translate("Upload_Window", "Choose perecentage of summary:"))
        self.label_3.setText(_translate("Upload_Window", "Choose amount of RBM's:"))
        self.SummarizeButton.setText(_translate("Upload_Window", "Summarize"))
        self.CheckButton.setText(_translate("Upload_Window", "Check Similar Summaries"))
        self.OpenVocabularyButton.setText(_translate("Upload_Window", "Open Local Vocabulary"))
        item = self.tableWidget.horizontalHeaderItem(0)
        item.setText(_translate("Upload_Window", "File Name"))
        self.OpenSummaryButton.setText(_translate("Upload_Window", "Open Summary"))
        self.OpenSource.setText(_translate("Upload_Window", "Open Source"))


# Similar Summaries window
###### This is the GUI and Controller of the Similar Summaries window

In [9]:
class Ui_Similar_Summaries(object):
    def setupUi(self, Similar_Summaries,list_Init,prevSummary):
        Similar_Summaries.setObjectName("Similar_Summaries")
        Similar_Summaries.resize(800, 600)
        self.centralwidget = QtWidgets.QWidget(Similar_Summaries)
        self.centralwidget.setObjectName("centralwidget")
        
        #self.labelpic = QtWidgets.QLabel(self.centralwidget)
        #self.labelpic.setGeometry(QtCore.QRect(0, -2, 830, 750))
        #self.labelpic.setStyleSheet("background-image: url(:/newPrefix/sumilarimg.jpg);")
        #self.labelpic.setText("")
        #self.labelpic.setObjectName("labelpic")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(2, 2, 1000, 900))
        self.label.setStyleSheet("background-image: url(:/newPrefix/sumilarimg.jpg);")
        self.label.setText("")
        self.label.setObjectName("label")
        
        self.HelpButton = QtWidgets.QPushButton(self.centralwidget)
        self.HelpButton.setGeometry(QtCore.QRect(680, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.HelpButton.setFont(font)
        self.HelpButton.setObjectName("HelpButton")
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(10)
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(85, 60, 651, 71))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(28)
        font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(60)
        font.setStrikeOut(False)
        self.label.setFont(font)
        self.label.setMouseTracking(False)
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(260, 110, 311, 61))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(20)
        font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(60)
        font.setStrikeOut(False)
        self.label_2.setFont(font)
        self.label_2.setMouseTracking(False)
        self.label_2.setObjectName("label_2")
        self.tableWidget = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget.setGeometry(QtCore.QRect(90, 170, 601, 271))
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(2)
        self.tableWidget.setRowCount(0)
        
        header = self.tableWidget.horizontalHeader()
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QtWidgets.QHeaderView.ResizeToContents)
        
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(0, item)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(1, item)
        self.showGraphButton = QtWidgets.QPushButton(self.centralwidget)
        self.showGraphButton.setGeometry(QtCore.QRect(590, 450, 101, 28))
        self.showGraphButton.setObjectName("showGraphButton")
        self.OpenSummaryButton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSummaryButton.setGeometry(QtCore.QRect(190, 450, 101, 28))
        self.OpenSummaryButton.setObjectName("OpenSummaryButton")
        self.OpenVocabularyButton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenVocabularyButton.setGeometry(QtCore.QRect(300, 450, 151, 28))
        self.OpenVocabularyButton.setObjectName("OpenVocabularyButton")
        self.FuseButton = QtWidgets.QPushButton(self.centralwidget)
        self.FuseButton.setGeometry(QtCore.QRect(460, 450, 121, 28))
        self.FuseButton.setObjectName("FuseButton")
        self.OpenSource = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSource.setGeometry(QtCore.QRect(90, 450, 91, 28))
        self.OpenSource.setObjectName("OpenSource")
        Similar_Summaries.setCentralWidget(self.centralwidget)
        
        self.tableIndex=0
        self.tableValue=""
        self.prevSummarySaved = prevSummary+"_Summary.txt"
        
        for list_item in list_Init:
            rowPosition = self.tableWidget.rowCount()
            self.tableWidget.insertRow(rowPosition)
            list_init_item=list_item.split()
            self.tableWidget.setItem(self.tableIndex,0, QTableWidgetItem(list_init_item[0]))
            self.tableWidget.setItem(self.tableIndex,1, QTableWidgetItem(list_init_item[1]))
            self.tableIndex=self.tableIndex+1
        self.retranslateUi(Similar_Summaries)
        QtCore.QMetaObject.connectSlotsByName(Similar_Summaries)
        if(self.tableValue==""):
            self.OpenSummaryButton.setDisabled(True)
            self.OpenVocabularyButton.setDisabled(True)
            self.FuseButton.setDisabled(True)
            self.showGraphButton.setDisabled(True)
            self.OpenSource.setDisabled(True)
            
        #Functions
        self.tableWidget.clicked.connect(self.TableContent)
        self.HelpButton.clicked.connect(self.Help_Func)
        self.FuseButton.clicked.connect(self.FuseSummaries)
        self.OpenSummaryButton.clicked.connect(self.openSummaryFile)
        self.OpenVocabularyButton.clicked.connect(self.openVocabFile)
        self.showGraphButton.clicked.connect(self.showGraphWindow)
        self.OpenSource.clicked.connect(self.OpenSourceFunc)
        
    def showGraphWindow(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_similar_graph()
        self.ui.setupUi(self.window,self.tableValue,self.prevSummarySaved)
        self.window.show()
    def openVocabFile(self):
        dictName=self.tableValue+"_Dictionary.txt"
        webbrowser.open(dictName)
    def openSummaryFile(self):
        SummaryName=self.tableValue+"_Summary.txt"
        webbrowser.open(SummaryName)
    def Help_Func(self):
        msg = QMessageBox()
        msg.setWindowTitle("Help")
        msg.setText("If you found a bug or want to ask any question "+"\nyou can contact us at:\n"
        + "Nofar Hazan- nofarhazan5@gmail.com\nDaniel Larionov-danielarionov@gmail.com ")
        x = msg.exec_()  # this will show our messagebox    
    def FuseSummaries(self):
        SummaryName=self.tableValue+"_Summary.txt"
        fuseSummariesFunc(SummaryName,self.prevSummarySaved)
    def TableContent(self):
        # #selected cell value.
        index=(self.tableWidget.selectionModel().currentIndex())
        self.OpenSummaryButton.setDisabled(False)
        self.OpenVocabularyButton.setDisabled(False)
        self.FuseButton.setDisabled(False)
        self.showGraphButton.setDisabled(False)
        self.OpenSource.setDisabled(False)
        print(index)
        self.tableValue=index.sibling(index.row(),index.column()).data()
         
    def OpenSourceFunc(self):
        flag=0
        SummaryName=self.tableValue+".txt"
        saved_Summaries=glob.glob("*.txt")
        summarySearch=self.tableValue+".txt"
        for file in saved_Summaries:
            if (file == summarySearch):
                webbrowser.open(SummaryName)
                flag=1
        if (flag==0):
            fileName, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select File", "", "Txt Files (*.txt)")
            webbrowser.open(fileName)   
            
    def retranslateUi(self, Similar_Summaries):
        _translate = QtCore.QCoreApplication.translate
        Similar_Summaries.setWindowTitle(_translate("Similar_Summaries", "Similar Summaries"))
        self.HelpButton.setText(_translate("Similar_Summaries", "Help"))
        self.label.setText(_translate("Similar_Summaries", "Automatic Text Summarization"))
        self.label_2.setText(_translate("Similar_Summaries", "Similar  Summaries"))
        item = self.tableWidget.horizontalHeaderItem(0)
        item.setText(_translate("Similar_Summaries", "File Name"))
        item = self.tableWidget.horizontalHeaderItem(1)
        item.setText(_translate("Similar_Summaries", "Similarity Score"))
        self.OpenSummaryButton.setText(_translate("Similar_Summaries", "Open Summary"))
        self.OpenVocabularyButton.setText(_translate("Similar_Summaries", "Open Local Vocabulary"))
        self.FuseButton.setText(_translate("Similar_Summaries", "Fuse Summaries"))
        self.showGraphButton.setText(_translate("Similar_Summaries", "Show Graph"))
        self.OpenSource.setText(_translate("Similar_Summaries", "Open Source"))
        
        

# Saved Summaries window
###### This is the GUI and Controller of the Saved Summaries window

In [10]:
class Ui_Saved_Summaries(object):
    def setupUi(self, Saved_Summaries):
        Saved_Summaries.setObjectName("Saved_Summaries")
        Saved_Summaries.resize(800, 600)
        self.Saved_Summaries_Back=Saved_Summaries
        self.centralwidget = QtWidgets.QWidget(Saved_Summaries)
        self.centralwidget.setObjectName("centralwidget")
        
        self.labelpic = QtWidgets.QLabel(self.centralwidget)
        self.labelpic.setGeometry(QtCore.QRect(0, 2, 1200, 641))
        self.labelpic.setStyleSheet("background-image: url(:/newPrefix/saveds.jpg);")
        self.labelpic.setText("")
        self.labelpic.setObjectName("labelpic")

        
        self.BackButton = QtWidgets.QPushButton(self.centralwidget)
        self.BackButton.setGeometry(QtCore.QRect(22, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.BackButton.setFont(font)
        self.BackButton.setObjectName("BackButton")
        self.HelpButton = QtWidgets.QPushButton(self.centralwidget)
        self.HelpButton.setGeometry(QtCore.QRect(680, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.HelpButton.setFont(font)
        self.HelpButton.setObjectName("HelpButton")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(85, 60, 651, 71))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(28)
        #font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(60)
        font.setStrikeOut(False)
        self.label.setFont(font)
        self.label.setMouseTracking(False)
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(260, 110, 311, 61))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(20)
        font.setBold(True)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(60)
        font.setStrikeOut(False)
        self.label_2.setFont(font)
        self.label_2.setMouseTracking(False)
        self.label_2.setObjectName("label_2")
        self.tableWidget = QtWidgets.QTableWidget(self.centralwidget)
        self.tableWidget.setGeometry(QtCore.QRect(90, 170, 601, 271))
        self.tableWidget.setObjectName("tableWidget")
        self.tableWidget.setColumnCount(1)
        self.tableWidget.setRowCount(0)
        item = QtWidgets.QTableWidgetItem()
        self.tableWidget.setHorizontalHeaderItem(0, item)
        self.tableWidget.setEditTriggers(QtWidgets.QAbstractItemView.NoEditTriggers)
        
        header = self.tableWidget.horizontalHeader()
        header.setSectionResizeMode(0, QtWidgets.QHeaderView.ResizeToContents)

        
        self.OpenSummaryBitton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSummaryBitton.setGeometry(QtCore.QRect(210, 450, 131, 28))
        self.OpenSummaryBitton.setObjectName("OpenSummaryBitton")
        self.CheckButton = QtWidgets.QPushButton(self.centralwidget)
        self.CheckButton.setGeometry(QtCore.QRect(520, 450, 171, 28))
        self.CheckButton.setObjectName("CheckButton")
        self.OpenVocabularyButton = QtWidgets.QPushButton(self.centralwidget)
        self.OpenVocabularyButton.setGeometry(QtCore.QRect(350, 450, 161, 28))
        self.OpenVocabularyButton.setObjectName("OpenVocabularyButton")
        
        self.OpenSource = QtWidgets.QPushButton(self.centralwidget)
        self.OpenSource.setGeometry(QtCore.QRect(90, 450, 111, 28))
        self.OpenSource.setObjectName("OpenSource")
        Saved_Summaries.setCentralWidget(self.centralwidget)
        #Variables
        self.tableIndex=0
        self.tableValue=""
        
        if(self.tableValue==""):
            self.OpenSummaryBitton.setDisabled(True)
            self.OpenVocabularyButton.setDisabled(True)
            self.CheckButton.setDisabled(True)
            self.OpenSource.setDisabled(True)
            
        #Functions
        self.retranslateUi(Saved_Summaries)
        QtCore.QMetaObject.connectSlotsByName(Saved_Summaries)
        self.Open_Saved_Summaries()
        self.BackButton.clicked.connect(self.Back_toMain_Func)
        self.HelpButton.clicked.connect(self.Help_Func)
        self.tableWidget.clicked.connect(self.TableContent)
        self.OpenSummaryBitton.clicked.connect(self.openSummaryFile)
        self.OpenVocabularyButton.clicked.connect(self.openVocabFile)
        self.CheckButton.clicked.connect(self.checkForSimiliarSummaries)
        self.OpenSource.clicked.connect(self.OpenSourceFunc)
        
    def checkForSimiliarSummaries(self):
        dictName=self.tableValue+"_Dictionary.txt"
        list_init=checkForSimilarSummaries(dictName)
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Similar_Summaries()
        self.ui.setupUi(self.window,list_init,self.tableValue)
        self.window.show()
        
    def OpenSourceFunc(self):
        flag=0
        SummaryName=self.tableValue+".txt"
        saved_Summaries=glob.glob("*.txt")
        summarySearch=self.tableValue+".txt"
        for file in saved_Summaries:
            if (file == summarySearch):
                webbrowser.open(SummaryName)
                flag=1
        if (flag==0):
            fileName, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select File", "", "Txt Files (*.txt)")
            webbrowser.open(fileName)    
                
        
    def openVocabFile(self):
        dictName=self.tableValue+"_Dictionary.txt"
        webbrowser.open(dictName)
    def openSummaryFile(self):
        SummaryName=self.tableValue+"_Summary.txt"
        webbrowser.open(SummaryName)           
    def TableContent(self):
        # #selected cell value.
        index=(self.tableWidget.selectionModel().currentIndex())
        self.OpenSummaryBitton.setDisabled(False)
        self.OpenVocabularyButton.setDisabled(False)
        self.CheckButton.setDisabled(False)
        self.OpenSource.setDisabled(False)
        print(index)
        self.tableValue=index.sibling(index.row(),index.column()).data()
    def Open_Saved_Summaries(self):
        saved_Summaries=glob.glob("*_Summary.txt")
        for file in saved_Summaries:
            rowPosition = self.tableWidget.rowCount()
            self.tableWidget.insertRow(rowPosition)
            file_name_fix=file.replace('_Summary.txt','')
            self.tableWidget.setItem(0,self.tableIndex, QTableWidgetItem(file_name_fix))
            self.tableIndex=self.tableIndex+1
    def Help_Func(self):
        msg = QMessageBox()
        msg.setWindowTitle("Help")
        msg.setText("If you found a bug or want to ask any question "+"\nyou can contact us at:\n"
        + "Nofar Hazan- nofarhazan5@gmail.com\nDaniel Larionov-danielarionov@gmail.com ")
        x = msg.exec_()  # this will show our messagebox
    def Back_toMain_Func(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Opening_Window()
        self.ui.setupUi(self.window)
        self.Saved_Summaries_Back.hide()
        self.window.show()
    def retranslateUi(self, Saved_Summaries):
        _translate = QtCore.QCoreApplication.translate
        Saved_Summaries.setWindowTitle(_translate("Saved_Summaries", "Saved Summaries"))
        self.BackButton.setText(_translate("Saved_Summaries", "Back"))
        self.HelpButton.setText(_translate("Saved_Summaries", "Help"))
        self.label.setText(_translate("Saved_Summaries", "Automatic Text Summarization"))
        self.label_2.setText(_translate("Saved_Summaries", "Saved  Summaries"))
        item = self.tableWidget.horizontalHeaderItem(0)
        item.setText(_translate("Saved_Summaries", "File Name"))
        self.OpenSummaryBitton.setText(_translate("Saved_Summaries", "Open Summary"))
        self.CheckButton.setText(_translate("Saved_Summaries", "Check Similar Summaries"))
        self.OpenVocabularyButton.setText(_translate("Saved_Summaries", "Open Local Vocabulary"))
        self.OpenSource.setText(_translate("OpenSource", "Open Source"))

# Main window 
###### This is the GUI and Controller of the Main window
###### The program starts from this window

In [ ]:
class Ui_Opening_Window(object):
    def setupUi(self, Opening_Window):
        Opening_Window.setObjectName("Opening_Window")
        Opening_Window.resize(800, 600)
        self.Opening_Window_back=Opening_Window
        font = QtGui.QFont()
        font.setPointSize(8)
        Opening_Window.setFont(font)
        Opening_Window.setStyleSheet("")
        self.centralwidget = QtWidgets.QWidget(Opening_Window)
        self.centralwidget.setObjectName("centralwidget")
        
        self.lable_pic = QtWidgets.QLabel(self.centralwidget)
        self.lable_pic.setGeometry(QtCore.QRect(-100, -70, 1200, 691))
        self.lable_pic.setStyleSheet("background-image: url(:/newPrefix/WhatsApp Image 2020-05-18 at 09.54.31.jpg);")
        self.lable_pic.setObjectName("lable_pic")
    
        self.CreateNewSummary = QtWidgets.QPushButton(self.centralwidget)
        self.CreateNewSummary.setGeometry(QtCore.QRect(260, 250, 281, 41))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.CreateNewSummary.setFont(font)
        self.CreateNewSummary.setObjectName("CreateNewSummary")
        self.ShowSaveSummaries = QtWidgets.QPushButton(self.centralwidget)
        self.ShowSaveSummaries.setGeometry(QtCore.QRect(260, 320, 281, 41))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.ShowSaveSummaries.setFont(font)
        self.ShowSaveSummaries.setObjectName("ShowSaveSummaries")
        self.lable = QtWidgets.QLabel(self.centralwidget)
        self.lable.setGeometry(QtCore.QRect(105, 120, 641, 121))
        font = QtGui.QFont()
        font.setFamily("Assistant")
        font.setPointSize(28)
        font.setBold(False)
        font.setItalic(False)
        font.setUnderline(False)
        font.setWeight(56)
        font.setStrikeOut(False)
        self.lable.setFont(font)
        self.lable.setMouseTracking(False)
        self.lable.setObjectName("lable")
        self.HelpButton = QtWidgets.QPushButton(self.centralwidget)
        self.HelpButton.setGeometry(QtCore.QRect(680, 20, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.HelpButton.setFont(font)
        self.HelpButton.setObjectName("HelpButton")
        Opening_Window.setCentralWidget(self.centralwidget)

        #Functions
        self.retranslateUi(Opening_Window)
        QtCore.QMetaObject.connectSlotsByName(Opening_Window)
        self.CreateNewSummary.clicked.connect(self.Open_US_Window)
        self.HelpButton.clicked.connect(self.Help_Func)
        self.ShowSaveSummaries.clicked.connect(self.Open_saved_Window)
        
    def Open_US_Window(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Upload_Window()
        self.ui.setupUi(self.window)
        self.Opening_Window_back.hide()
        self.window.show()
        
    def Open_saved_Window(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_Saved_Summaries()
        self.ui.setupUi(self.window)
        self.Opening_Window_back.hide()
        self.window.show()   
        
    def Help_Func(self):
        msg = QMessageBox()
        msg.setWindowTitle("Help")
        msg.setText("If you found a bug or want to ask any question "+"\nyou can contact us at:\n"
        + "Nofar Hazan- nofarhazan5@gmail.com\nDaniel Larionov-danielarionov@gmail.com ")
        x = msg.exec_()  # this will show our messagebox
        
    
    def retranslateUi(self, Opening_Window):
        _translate = QtCore.QCoreApplication.translate
        Opening_Window.setWindowTitle(_translate("Opening_Window", "Main Window"))
        self.CreateNewSummary.setText(_translate("Opening_Window", "Create New Summary"))
        self.ShowSaveSummaries.setText(_translate("Opening_Window", "Show Saved Summaries"))
        self.lable.setText(_translate("Opening_Window", "Automatic Text Summarization"))
        self.HelpButton.setText(_translate("Opening_Window", "Help"))
        
        self.lable_pic.setText(_translate("Opening_Window", "TextLabel"))
    
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Opening_Window = QtWidgets.QMainWindow()
    ui = Ui_Opening_Window()
    ui.setupUi(Opening_Window)
    Opening_Window.show()
    sys.exit(app.exec_())

# End